In [5]:
import tensorflow as tf
from glob import glob
import cv2
import numpy as np
from tqdm import tqdm

# Dataset
## Get

In [38]:
IMG_TARGET_SIZE = (224, 224)
DATA_FOLDER = "data/Image_Classification/dataset_25061716"

dataset = []
labels = []
labels_name = []

classes = glob(DATA_FOLDER + "/*")

# get all classes
for class_idx, class_folder in tqdm(list(enumerate(classes))):
    labels_name.append(class_folder.split("/")[-1])
    
    # get all images in class folder
    for image_file in glob(class_folder + "/*.jpg"):
        image = cv2.imread(image_file)
        image = cv2.resize(image, IMG_TARGET_SIZE)
        
        dataset.append(image)
        
        # hot encoding labels
        label = np.zeros(len(classes))
        label[class_idx] = 1
        labels.append(label)

dataset = np.array(dataset)
labels = np.array(labels)

print(f"data shape: {dataset.shape}, labels shape: {labels.shape}")

100%|██████████| 20/20 [00:03<00:00,  6.35it/s]


data shape: (3248, 224, 224, 3), labels shape: (3248, 20)


## Split

In [43]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.3, random_state=25061716)
test_dataset, val_dataset, test_labels, val_labels = train_test_split(test_dataset, test_labels, test_size=0.3, random_state=25061716)

print(f"train data shape: {train_dataset.shape}, train labels shape: {train_labels.shape}")
print(f"test data shape: {test_dataset.shape}, test labels shape: {test_labels.shape}")
print(f"val data shape: {val_dataset.shape}, val labels shape: {val_labels.shape}")

train data shape: (2273, 224, 224, 3), train labels shape: (2273, 20)
test data shape: (682, 224, 224, 3), test labels shape: (682, 20)
val data shape: (293, 224, 224, 3), val labels shape: (293, 20)


In [44]:
from tensorflow.keras.applications import resnet

model = resnet.ResNet50(include_top=True, weights=None, input_shape=(IMG_TARGET_SIZE[0], IMG_TARGET_SIZE[1], 3), classes=len(classes))
print(model.summary())

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_3[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,628,692 (90.14 MB)

 Trainable params: 23,575,572 (89.93 MB)

 Non-trainable params: 53,120 (207.50 KB)

None


In [46]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [48]:
history = model.fit(train_dataset, train_labels, validation_data=(val_dataset, val_labels), epochs=10, batch_size=32)

Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 407s 5s/step - accuracy: 0.0816 - loss: 4.2219 - val_accuracy: 0.0614 - val_loss: 7.2740
Epoch 2/10
36/72 ━━━━━━━━━━━━━━━━━━━━ 3:33 6s/step - accuracy: 0.2078 - loss: 2.7640

In [1]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


NameError: name 'history' is not defined

In [2]:
model.save("model.h5")

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

predicted_labels = model.predict(test_dataset)
true_labels = np.argmax(test_labels, axis=1)

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy}")
